# dbachecks PR check on Beard NUC - MultipleInstances

Rob uses this process to make sure that changes will work in an AD estate with a variety of versions

First we must check the status of the local source control and ensure that we are in the PR branch

In [1]:
cd D:\OneDrive\Documents\GitHub\dbachecks
    git status

On branch pr/tboggiano/756-1
Your branch is ahead of 'tboggiano/querystorenew' by 116 commits.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   RELEASE.md
	modified:   checks/Database.Tests.ps1
	modified:   internal/configurations/configuration.ps1

no changes added to commit (use "git add" and/or "git commit -a")


First we need to import the module from source control - I normally use verbose to see how it is going


In [2]:
Import-Module .\dbachecks.psd1
$null= Reset-DbcConfig

Excellent, that worked. Now to run some checks against a single instance  

I don't care if the test pass or fail, I am only interested in any 

> Describe block has Error

or

> Error in Script

type failures

Lets check the check and config have been populated

In [5]:
Get-DbcCheck ContainedDBSQLAuth | fl
Get-DbcCheck QueryStoreEnabled | fl


Group       : Database
Type        : Sqlinstance
UniqueTag   : ContainedDBSQLAuth
AllTags     : ContainedDBSQLAuth, CIS, Database
Config      : app.sqlinstance skip.security.ContainedDBSQLAuth 
Description : Tests to see a contained database contains sql authenticated users.



Group       : Database
Type        : Sqlinstance
UniqueTag   : QueryStoreEnabled
AllTags     : QueryStoreEnabled, Medium, Database
Config      : app.sqlinstance database.querystore.excludedb 
Description : Tests to see Query Store is enabled on each database.




In [7]:
Get-DbcConfig  database.querystore.excludedb |fl
Get-DbcConfig skip.security.ContainedDBSQLAuth |fl


Name        : database.querystore.excludedb
Value       : {}
Description : A List of databases that we do not want to check for Query Store



Name        : skip.security.ContainedDBSQLAuth
Value       : True
Description : Skips the scan for if a contained database as sql authenticated users




Multiple Instances at parameter

In [9]:
$Sqlinstances = (Get-VM -ComputerName BEARDNUC, BEARDNUC2 -Name *SQL* | Where State -eq 'Running').Name

Invoke-DbcCheck -SqlInstance $Sqlinstances -Check ContainedDBSQLAuth,QueryStoreEnabled

    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'D:\OneDrive\Documents\GitHub\dbachecks\checks\Database.Tests.ps1' with Tags ContainedDBSQLAuth', 'QueryStoreEnabled

Executing script D:\OneDrive\Documents\GitHub\dbachecks\checks\Database.Tests.ps1

  Describing Contained Database SQL Authenticated Users

    Context Testing contained database to see if sql authenticated users exist on SQL2005Ser2003
      [!] AdventureWorks on SQL2005Ser2003 should have no sql authenticated users, is skipped 0ms
      [!] AdventureWorksDW on SQL2005Ser2003 should have no sql authenticated users, is skipped 0ms
      [!] Aerosmith on SQL2005Ser2003 should have no sql authenticated users, is skipped 0ms
      [!] BlackSabbath on SQL2005Ser2003 should have no sql authenticated users, is skipped 0ms
      [!] DBA-Admin on SQL2005Ser2003 should have no sql authenticated 

Set the config and run again

In [ ]:
$Sqlinstances = (Get-VM -ComputerName BEARDNUC, BEARDNUC2 -Name *SQL* | Where State -eq 'Running').Name

Set-DbcConfig -Name skip.security.sadisabled -Value $false
Invoke-DbcCheck -SqlInstance $Sqlinstances -Check SaDisabled

In [ ]:
Get-DbaServerRole -SQLInstance sql2019n5 -ServerRole "sysadmin"

In [ ]:
@(Get-DbaLogin -SQLInstance sql2019n5 -Type SQL | Where-Object { $_.PasswordPolicyEnforced -eq $false -and $_.IsDisabled -eq $false}).Count

In [ ]:
@(Get-DbaLogin -SqlInstance sql2019n5 -Login sa).Count